## Task: Use K-fold CV to select `dly`

In [1]:
import numpy as np
import pickle
from sklearn.linear_model import LinearRegression
from sklearn.metrics import r2_score, mean_squared_error
from sklearn.model_selection import train_test_split, KFold

In [2]:
with open('example_data_s1.pickle', 'rb') as fp:
    X,y = pickle.load(fp)

tsamp = 0.05  
nt, nneuron = X.shape
nout = y.shape[1]
ttotal = nt*tsamp

In [3]:
nred = 6000

Xred = X[:nred]
yred = y[:nred]

In [4]:
def create_dly_data(X,y,dly):
    """
    Create delayed data
    """    
    n,p = X.shape
    Xdly = np.zeros((n-dly,(dly+1)*p))
    for i in range(dly+1):
        Xdly[:,i*p:(i+1)*p] = X[dly-i:n-i,:]
    ydly = y[dly:]
    
    return Xdly, ydly

In [5]:
dmax = 15

Xdly, ydly = create_dly_data(Xred,yred,dmax)

In [6]:
dtest_list = np.arange(0, dmax+1)
nd = len(dtest_list)
nfold = 10

Use K-fold CV with `nfold=10` to find the optimal delay, for all the values of delay in `dtest_list`.

In [7]:
#grade (write your code in this cell and DO NOT DELETE THIS LINE)

#  Create a k-fold object
kf = KFold(n_splits=nfold, shuffle=False)
 
# Initialize a matrix Rsq to hold values of the R^2 across the model orders and folds.
Rsq = np.zeros((nd,nfold))
 
# Loop over the folds
for i, idx_split in enumerate(kf.split(Xdly)):
    
    # Get the training and validation data in the split
    idx_tr, idx_val = idx_split
    
    for it, dtest in enumerate(dtest_list):
        # Select the appropriate subset of columns of Xdly
        X_dtest = Xdly[:, :X.shape[1]*(dtest+1)]
        
        # Split the data (X_dtest, ydly) into training and validation using idx_tr and idx_val
        Xtr = X_dtest[idx_tr, :]
        ytr = ydly[idx_tr]
        Xval = X_dtest[idx_val, :]
        yval = ydly[idx_val]
        
        # Fit linear regression on training data
        reg = LinearRegression().fit(Xtr, ytr)
        
        # Measure the R2 on validation data and store in the matrix Rsq
        yhat_val = reg.predict(Xval)
        Rsq[it, i] = r2_score(yval, yhat_val)

Write code to find the delay that has the best validation R2. Get the
best delay according to the "best R2" rule, and save it in `d_opt`.


In [10]:
#grade (write your code in this cell and DO NOT DELETE THIS LINE)
Rsq_mean = np.mean(Rsq, axis=1)
d_opt_index = np.argmax(Rsq_mean)
d_opt = dtest_list[d_opt_index]
d_opt

8

Now write code to find the best delay using the one SE rule (i.e. find
the simplest model whose validation R2 is within one SE of the model
with the best R2). Get the best delay according to the "one SE
rule", and save it in `d_one_se`.


In [11]:
#grade (write your code in this cell and DO NOT DELETE THIS LINE)
# Calculate the standard error of the R^2 values
Rsq_se = np.std(Rsq, axis=1) / np.sqrt(nfold)

# Determine the threshold
threshold = Rsq_mean[d_opt_index] - Rsq_se[d_opt_index]

# Find the simplest model that is within one SE of the best model
d_one_se_index = np.where(Rsq_mean >= threshold)[0][0]
d_one_se = dtest_list[d_one_se_index]
d_one_se

6